In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


import pinecone
from pinecone import ServerlessSpec
import time

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY") #langchain searches for this key and loads it automatically
os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY") #langchain searches for this key and loads it automatically

In [2]:
# Load PDF File
loader = PyPDFLoader('./assets/srp-covid-19-6month.pdf')
pdf = loader.load()

In [ ]:
# Pinecone index name
index_name = "rag-project"

# embeddings using OpenAI "text-embedding-ada-002"
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create the vector store
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name, 
    embedding=embed_model, 
    text_key="text", # the key containing the text chunks in the meta data
    namespace="all-users", # optional if you didn't set a name space
    pool_threads=4
)

### Using a Simple QA Retriever

In [16]:
# Creat the QA Chain
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever()
)

query = "When was thef first covid case?"

qa.invoke(query)



{'query': 'When was thef first covid case?',
 'result': 'The first known cases of COVID-19 were reported in December 2019 in Wuhan, China.'}

### Using a Retrieval Chain With A Custom Prompt Template

##### *Using OllamaLLM llama2*

In [24]:
# Import Ollama LLM
from langchain_ollama import OllamaLLM

# Load Ollama LAMA2 LLM Model
llm = OllamaLLM(model='llama2')

# Import Ollama LLM
from langchain_openai import ChatOpenAI
# Load Ollama LAMA2 LLM Model
llm = ChatOpenAI(model='gpt-4o', temperature=0)

# Design Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    You are an AI assistant. Use only the provided context to answer the question.
    If the answer is unknown, say "I don't know."
    <context>
    {context}
    </context>

    Question: {input}
    """
)


# Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)


# Retrival
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever = vectorstore.as_retriever(), combine_docs_chain=document_chain)

query = "When was thef first covid case?"
response = retrieval_chain.invoke({"input": query})

print(response)

{'input': 'When was thef first covid case?', 'context': [Document(id='6', metadata={}, page_content='1 2\n \nCOVID-19: February–June progress report\n  \n    COVID-19: February–June progress report\nFOREWORD\nMore than six months since the world first learned of what we now call COVID-19, the time is \nright to take stock of where we are in the outbreak and how the world has responded. \nThe pandemic has already taken a terrible toll. By the end of June 2020, WHO had received \nreports of almost 10 million cases and half a million lives lost. The pandemic continues to \naccelerate; at the current rate, cases are doubling around every six weeks. We are facing a \nmoment of great danger. We can only prevail if we stand together in global solidarity. \nCOVID-19 will always take the path of least resistance. We know that when countries take \na comprehensive approach based on fundamental public health measures and a whole-\nof-society approach, COVID-19 can be brought under control, saving

##### *Using OpenAI's GPT-4o*

In [ ]:
# Import ChaptOpenAI
from langchain_openai import ChatOpenAI
# Load GPT-4o Model
llm = ChatOpenAI(model='gpt-4o', temperature=0)

# Design Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    You are an AI assistant. Use only the provided context to answer the question.
    If the answer is unknown, say "I don't know."
    <context>
    {context}
    </context>

    Question: {input}
    """
)


# Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)


# Retrival
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever = vectorstore.as_retriever(), combine_docs_chain=document_chain)

query = "When was thef first covid case?"
response = retrieval_chain.invoke({"input": query})

print(response)